In [1]:
import openpyxl as pyxl
import pandas as pd
import numpy as np
from gspread.utils import a1_to_rowcol, rowcol_to_a1

import shutil, os

import re
from itertools import product
# we only need the function datetime.datetime.now; we can now reference it as dt.now
from datetime import datetime as dt

import sqlite3 as sql

In [2]:
db_filename = 'my-budget-dev-v1.sqlite'

%run nuclear_option.py $db_filename
%run database_setup.py $db_filename

db = sql.connect(db_filename)

Creating an an empty database as "my-budget-dev-v1.sqlite"
Completed creating the database scheme. It looks as follows:


,name
0,money_pots
1,event_types
2,budget_pots
3,money_events
4,payments
5,recievings
6,transfers
7,budget_events
8,event_groups
9,event_in_group


The table money_pots: 


,key,description,liquid
0,KG,gemeinsames Konto,Yes
1,KE,Extrakonto zum gemeinsamen Konto,Yes
2,KM,Konto Max,Yes
3,KP,Konto Paul,Yes
4,KB,Konto Bundesbank,Yes
5,KC,Consorsbankkonto,No
6,BM,Bargeld Max,Yes
7,BP,Bargeld Paul,Yes
8,CB,Chipkarte Bundesbank,Semi
9,CT,Chipkarte Trianon,Semi


The table event_types: 


...

,group_id,event_id


The table database_event_types: 


,type,description
0,Erstellung,Erstellung eines Eintrages
1,Update,Hinzufügen von Information
2,Korrektur,Korrigieren eines Eintrages
3,Löschung,Löschen eines Eintrages


The table database_events: 


,id,type,date,description


In [3]:
excel_filename = 'GemeinsameBilanzierung_16_17__dev.xlsx'

current_sheet = 'August'
comment_sheet = 'august_'

# We take here the option data_only since for now we are not interested in the expression
wb = pyxl.load_workbook(excel_filename, data_only=True)


august = wb.get_sheet_by_name(current_sheet)
august_ = wb.copy_worksheet(august)
august_.title = comment_sheet
august_.sheet_state = 'hidden'

In [4]:
def list_from_range_string(range_string) :
    '''Extract all individual cell names from a excel range.
    
    Keyword arguments:
        range_string - The excel expression for the range
    
    Example:
        If range_string == 'A1:B3' then the list ['A1', 'B1', 'A2', 'B2', 'A3', 'B3'] is returned
    '''
    colon_position = range_string.find(':')
    if colon_position == -1 :
        raise
    first_cell = range_string[:colon_position]
    last_cell = range_string[colon_position+1:]
    
    first_row, first_col = a1_to_rowcol(first_cell)
    last_row, last_col = a1_to_rowcol(last_cell)
    
    return [rowcol_to_a1(i,j) for i,j in product(range(first_row, last_row+1), range(first_col, last_col+1))]

In [5]:
def generate_id(date) :
    '''Generate a new unique ID in the budgeter on the database. An ID is an integer with 8 digits, where 
    the first digit are based on the date and the last two digits are a serial number.
    
    Keyword arguments:
        date - the date on which the ID should be based
        
    Example: 
        If date==DateTime('2017-08-17') and the database contains the IDs 
        2017081701, 2017081702, 2017081703 and 2017081705, then the id 2017081704 is returned.
    
    Exceptions:
        IndexError - if all 99 possible serial numbers (01-99) have already been distributed.
        
        N.B. Could also Except, if the Database Call raises an exception.
    '''
    date_int = int(date.strftime('%Y%m%d00'))
    crsr = db.cursor()
    crsr.execute('SELECT id FROM money_events WHERE id BETWEEN {} AND {}'.format(date_int, date_int + 99))
    results = [row[0] for row in crsr.fetchall()]

    current_id = date_int + 1
    while current_id in results :
        current_id += 1
    if current_id > date_int + 99 :
        raise IndexError('Encountered to many ids for the date {}'.format(date))

    return current_id

In [6]:
def date_convert(item) :
    '''Try to get a DateTime from the excel cell, independent if it is represented by an integer, i.e. in 
    the native excel date format, or a string representation
    '''
    try :
          return pyxl.utils.datetime.from_excel(item)
    except :
        if type(item) is str :
            return pd.to_datetime(item)
        else :
            return pd.Timestamp(item)
        #pass
            

def get_df_by_range(sheet, first_cell, last_cell, date_cols=None) :
    '''Read a given range on the given sheet and return a DataFrame containing the data.
    
    Keyword arguments:
        sheet      - a openpyxl sheet object which is to be read
        first_cell - the top left cell of the range to be read; in excel cell notation
        last_cell  - the bottom right cell of the range to be read; in excel cell notation
        date_cols  - a column (or list of columns) which are assumed to contain dates and shall be returned
                     as pandas Timestamp object; can be given either as number (starting in zero) or a excel
                     column name
                     
        ToDo : Look if starting in zero is correct 
    '''
    data_rows = [[cell.value for cell in row] + ['{0}:{1}'.format(row[0].coordinate, row[-1].coordinate)]
        for row in august[first_cell:last_cell]]

    df = pd.DataFrame(data_rows)
    new_index = df.iloc[:,range(len(df.columns)-1)].dropna(how='all').index
    if date_cols is not None and type(date_cols) is int :
        df.iloc[:,date_cols] = df.iloc[:,date_cols].apply(date_convert).copy()
    elif type(date_cols) is str : 
        df.loc[:,date_cols] = df.loc[:,date_cols].apply(date_convert).copy()
    elif type(date_cols) is list :
        for col in date_cols :
            if type(col) is int :
                df.iloc[:,col] = df.iloc[:,col].apply(date_convert).copy()
            if type(col) is str : 
                df.loc[:,col] = df.loc[:,col].apply(date_convert).copy()
    return df.loc[new_index]

In [7]:
def put_comment_into_excel(sheet, cells, comment_text) :
    comment = pyxl.comments.Comment(comment_text, 'budgeter')
    if type(cells) is list :
        for cell in cells :
            sheet[cell].comment = comment
    elif type(cells) is str :
        sheet[cells].comment = comment

In [14]:
def put_payment_into_database_autogenerated(date, event_type, description, amount, 
                                            money_pot, budget_pot, comment) :
    the_id = generate_id(date)
    
    crsr = db.cursor()
    # ToDo : Hier machen wir die simplifizierende Annahme, dass alle Konto-Events Kartenzahlungen sind
    crsr.execute('''INSERT INTO money_events VALUES ({}, "{}", "{}", date("{}"));
        '''.format(the_id, event_type, description, date))
    crsr.execute('''INSERT INTO payments VALUES ({}, "{}", {}, NULL, NULL);
        '''.format(the_id,  money_pot, amount))
    crsr.execute('''INSERT INTO budget_events VALUES ({}, "{}", {}, NULL, NULL);
        '''.format(the_id, budget_pot, amount))
    crsr.execute('''INSERT INTO database_events VALUES ({}, "{}", "{}", "{}");
        '''.format(the_id, 'Erstellung', dt.now().strftime('%Y-%m-%d'), comment))
    db.commit()
    
    return the_id

def put_transfer_into_database_autogenerated(date, event_type, description, amount, 
                                             money_pot_source, money_pot_sink, 
                                             effect_date = None, comment = '') :
    the_id = generate_id(date)
    
    crsr = db.cursor()
    # ToDo : Hier machen wir die simplifizierende Annahme, dass alle Konto-Events Kartenzahlungen sind
    crsr.execute('''INSERT INTO money_events VALUES ({}, "{}", "{}", date("{}"));
        '''.format(the_id, event_type, description, date))
    crsr.execute('''INSERT INTO payments VALUES ({}, "{}", "{}", {}, {}, NULL, NULL);
        '''.format(the_id,  money_pot_source, money_pot_sink, amount,
                   'NULL' if effect_date is None else '"{}"'.format(effect_date)))
    crsr.execute('''INSERT INTO database_events VALUES ({}, "{}", "{}", "{}");
        '''.format(the_id, 'Erstellung', dt.now().strftime('%Y-%m-%d'), comment))
    db.commit()
    
    return the_id

In [15]:
def autogenerate_database_comment(filename, range_, comment_sheet) :
    return 'This entry was automatically generated from the excel file ' + \
           '{}. It is based on the cells {}. A note has been '.format(
               filename, range_) + \
           'added to the respective cells in the sheet {}.'.format(comment_sheet)

def autogenerate_database_comment_two_excel_ranges(filename, range1, range2, 
                                                   comment_sheet) :
    return 'This entry was automatically generated from the excel file ' + \
           '{}. It is based on the cells {} and {}. A note has been '.format(
               filename, range1, range2) + \
           'added to the respective cells in the sheet {}.'.format(comment_sheet)

def autogenerate_excel_comment(date, db_filename, the_id) :
    return 'On {} this cell was automatically read and '.format(date) + \
           'inserted into the database {}. The id ofthe entry is {}.'.format(
               date, db_filename, the_id)

In [10]:
budgeting = get_df_by_range(august_, 'A6', 'D130', 2)
budgeting.columns = ['budget_type', 'description', 'date', 'amount', 'excel_range']

col_titles = ['description', 'date', 'amount', 'excel_range']

max_bargeld = get_df_by_range(august_, 'H7', 'J130', 1)
max_bargeld.columns = col_titles
max_bargeld['money_pot'] = 'BM'

paul_bargeld = get_df_by_range(august_, 'K7', 'M130', 1)
paul_bargeld.columns = col_titles
paul_bargeld['money_pot'] = 'BP'

konto = get_df_by_range(august_, 'N7', 'P130', 1)
konto.columns = col_titles
konto['money_pot'] = 'KG'

conjoined = pd.concat([max_bargeld, paul_bargeld, konto])

In [11]:
all_info = pd.merge(budgeting, conjoined, how='outer', 
                    on=['description', 'date', 'amount'], indicator=True)

all_info['treated'] = 'No'

In [12]:
miete_info = all_info.loc[all_info.description == 'Miete'
                    ].loc[all_info.amount == -568
                    ].loc[all_info.treated != 'Yes']
for index, row in miete_info.iterrows() :
    database_comment = autogenerate_database_comment_two_excel_ranges(
        excel_filename, current_sheet + '!' + row['excel_range_x'],
        current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Dauerauftrag', 
                row['description'], row['amount'], row['money_pot'], 
                row['budget_type'], database_comment)
    
    crsr = db.cursor()
    crsr.execute('''INSERT INTO event_in_group VALUES (1, {});'''.format(the_id))
    db.commit()
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x']) + \
            list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [13]:
## Putting all the Info which had matching info on the left and the right hand of the budget sheet into
## the database

full_match = all_info[all_info['_merge'] == 'both'].loc[all_info['treated'] == 'No']


for index, row in full_match.iterrows() :
    database_comment = autogenerate_database_comment_two_excel_ranges(
        excel_filename, current_sheet + '!' + row['excel_range_x'],
        current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 
                 'Kartenzahlung' if row['money_pot'] == 'KG' else 'Barzahlung',
                 row['description'], row['amount'], row['money_pot'], 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x']) + \
            list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'
        
for table in ["money_events", "payments", "budget_events"] :
    print('The table {}: '.format(table))
    display(pd.read_sql_query('SELECT * FROM {};'.format(table), db))

The table money_events: 


,id,type,description,date
0,2017073101,Kartenzahlung,Monatskarte FFM,2017-07-31
1,2017073102,Kartenzahlung,Aldi,2017-07-31
2,2017073103,Kartenzahlung,Netto,2017-07-31
3,2017080101,Dauerauftrag,Miete,2017-08-01
4,2017080102,Kartenzahlung,Berufsunfähigkeitsversicherung,2017-08-01
5,2017080103,Kartenzahlung,Haftpflichtversicherung,2017-08-01
6,2017080104,Kartenzahlung,Rewe,2017-08-01
7,2017080105,Kartenzahlung,Rossmann,2017-08-01
8,2017080106,Kartenzahlung,DM,2017-08-01
9,2017080107,Kartenzahlung,DM,2017-08-01


The table payments: 


,id,money_pot,amount,additional_description,effect_date
0,2017080101,KG,-568.00,None,None
1,2017080201,KG,-450.00,None,None
2,2017080102,KG,-49.05,None,None
3,2017083001,KG,-51.00,None,None
4,2017080103,KG,-7.50,None,None
5,2017081001,KG,-7.99,None,None
6,2017081501,KG,-4.99,None,None
7,2017082501,KG,-4.99,None,None
8,2017073101,KG,-87.40,None,None
9,2017080301,KG,-39.80,None,None


The table budget_events: 


,id,budget_pot,amount,additional_description,budget_effet_date
0,2017080101,R,-568.00,None,None
1,2017080201,R,-450.00,None,None
2,2017080102,R,-49.05,None,None
3,2017083001,R,-51.00,None,None
4,2017080103,R,-7.50,None,None
5,2017081001,R,-7.99,None,None
6,2017081501,G,-4.99,None,None
7,2017082501,G,-4.99,None,None
8,2017073101,R,-87.40,None,None
9,2017080301,G,-39.80,None,None


In [ ]:
display(all_info)

In [ ]:
display(all_info[all_info['description'].str.contains('Schatulle')])

for direction in ['an', 'aus'] :
    descriptor = 'Geld ' + direction + ' Schatulle'
    schatulle_info = all_info[all_info['description'] == descriptor]

for index, row in schatulle_info.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    i = 0
    while True :
        the_date = all_info.loc[index+i, 'date']
        if str(the_date) == 'NaT' :
            the_date = all_info.loc[index-i, 'date']
            if str(the_date) == 'NaT' :
                i = i + 1
                continue
        
        break
        
       ### date, event_type, description, amount, 
       ### money_pot_source, money_pot_sink, 
       ### effect_date = None, comment = ''
            
    the_id = put_transfer_into_database_autogenerated(
        row['date'], 'Bargeldtransger', row['description'], row['amount'], 
        row['money_pot'], 'SB', database_comment)
    
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [ ]:
project_dir = '/Users/max/budgeter/'
template_file = './templates/simple_payment_list.xlsx'
temp_name = './present_left_side.xlsx'

os.chdir(project_dir)
shutil.copy(template_file, temp_name)

#print('Presenting the unmatched entries on the left side of the balance sheet')
#os.system('open ' + temp_name)

In [ ]:
#left_side_info = all_info[all_info['_merge'] == 'left_only']

# Aus einem mir unerfindlich grund speichert er das 'unhiding' der Spalte G erst nach 2 mal speichern

wb_temp = pyxl.load_workbook(temp_name)
active_sheet = wb_temp.active
print(active_sheet['A1'].value)
print(active_sheet.column_dimensions['G'].hidden)
active_sheet.column_dimensions['G'].hidden = False
wb_temp.save(temp_name)

wb_temp = pyxl.load_workbook(temp_name)
active_sheet = wb_temp.active




wb_temp.save(temp_name)

os.system('open ' + temp_name)

#for index, row in left_side_info.iterrows() :
    

In [ ]:
wb.save('GemBil.xlsx')

In [ ]:
db.close()

In [ ]:
display(left_side_info[0:20])